# 1. Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 2. Importando os csv's

In [2]:
train_data = pd.read_csv('train.csv', usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'], dtype={'store_nbr':'category', 'family': 'category', 'sales':'float32', 'onpromotion': 'uint64'}, parse_dates=['date'])
holidays_data = pd.read_csv('holidays_events.csv', usecols=['date', 'type', 'locale', 'locale_name', 'transferred'])
oil_data = pd.read_csv('oil.csv')
stores_data = pd.read_csv('stores.csv')
transactions_data = pd.read_csv('transactions.csv')

In [3]:
train_data['date'] = pd.to_datetime(train_data['date'])

# Converter a data em componentes numéricos
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day'] = train_data['date'].dt.day
train_data['day_of_week'] = train_data['date'].dt.day_of_week
train_data['store_nbr'] = train_data['store_nbr'].astype(int)

In [4]:
train_data = train_data.drop(['date'], axis=1)

In [5]:
train_data

,store_nbr,family,sales,onpromotion,year,month,day,day_of_week
0,1,AUTOMOTIVE,0.000000,0,2013,1,1,1
1,1,BABY CARE,0.000000,0,2013,1,1,1
2,1,BEAUTY,0.000000,0,2013,1,1,1
3,1,BEVERAGES,0.000000,0,2013,1,1,1
4,1,BOOKS,0.000000,0,2013,1,1,1
...,...,...,...,...,...,...,...,...
3000883,9,POULTRY,438.132996,0,2017,8,15,1
3000884,9,PREPARED FOODS,154.552994,1,2017,8,15,1
3000885,9,PRODUCE,2419.729004,148,2017,8,15,1
3000886,9,SCHOOL AND OFFICE SUPPLIES,121.000000,8,2017,8,15,1


In [6]:
train_data['day_of_week'].value_counts()

day_of_week
1    431244
5    429462
0    429462
2    427680
3    427680
4    427680
6    427680
Name: count, dtype: int64

# 3. Prevendo as vendas totais de cada loja

In [7]:
store_sales = train_data.groupby(['store_nbr'])['sales'].sum().reset_index()
store_sales

,store_nbr,sales
0,1,14145013.00
1,2,21557388.00
2,3,50481912.00
3,4,18909700.00
4,5,15592405.00
5,6,25183680.00
6,7,26952050.00
7,8,30494286.00
8,9,26409442.00
9,10,9613906.00


In [8]:
store_sales_per_date = train_data.groupby(['store_nbr', 'year', 'month', 'day', 'day_of_week'])['sales'].sum().reset_index()
store_sales_per_date

,store_nbr,year,month,day,day_of_week,sales
0,1,2013,1,1,1,0.000000
1,1,2013,1,2,2,7417.147949
2,1,2013,1,3,3,5873.244141
3,1,2013,1,4,4,5919.878906
4,1,2013,1,5,5,6318.785156
...,...,...,...,...,...,...
90931,54,2017,8,11,4,8513.833984
90932,54,2017,8,12,5,9139.677734
90933,54,2017,8,13,6,14246.828125
90934,54,2017,8,14,0,11882.994141


In [9]:
train, test = train_test_split(store_sales_per_date, test_size=0.2, random_state=42, shuffle=False)

model = LinearRegression()
features = ['store_nbr', 'year', 'month', 'day', 'day_of_week']
model.fit(train[features], train['sales'])
predictions = model.predict(test[features])
mse = mean_squared_error(test['sales'], predictions)
print(f'MSE: {mse}')

MSE: 376035441.55655533


# 4. Prevendo a venda de cada categoria de produto em cada loja

In [10]:
family_per_store_sales = train_data.groupby(['store_nbr', 'family'])['sales'].sum().reset_index()

family_per_store_sales

C:\Users\wpett\AppData\Local\Temp\ipykernel_26648\4292188311.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  family_per_store_sales = train_data.groupby(['store_nbr', 'family'])['sales'].sum().reset_index()


,store_nbr,family,sales
0,1,AUTOMOTIVE,5.475000e+03
1,1,BABY CARE,0.000000e+00
2,1,BEAUTY,4.056000e+03
3,1,BEVERAGES,2.673769e+06
4,1,BOOKS,2.110000e+02
...,...,...,...
1777,54,POULTRY,8.888981e+04
1778,54,PREPARED FOODS,1.030900e+05
1779,54,PRODUCE,5.961803e+05
1780,54,SCHOOL AND OFFICE SUPPLIES,1.023000e+03


In [11]:
family_per_store_sales = pd.get_dummies(family_per_store_sales, columns=['family'])
family_per_store_sales

,store_nbr,sales,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,1,5.475000e+03,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,0.000000e+00,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,4.056000e+03,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,2.673769e+06,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,2.110000e+02,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,54,8.888981e+04,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1778,54,1.030900e+05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1779,54,5.961803e+05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1780,54,1.023000e+03,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [12]:
family_per_store_sales_per_day = train_data.groupby(['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week'])['sales'].sum().reset_index()
family_per_store_sales_per_day

C:\Users\wpett\AppData\Local\Temp\ipykernel_26648\3650825506.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  family_per_store_sales_per_day = train_data.groupby(['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week'])['sales'].sum().reset_index()


,store_nbr,family,year,month,day,day_of_week,sales
0,1,AUTOMOTIVE,2013,1,1,0,0.0
1,1,AUTOMOTIVE,2013,1,1,1,0.0
2,1,AUTOMOTIVE,2013,1,1,2,0.0
3,1,AUTOMOTIVE,2013,1,1,3,0.0
4,1,AUTOMOTIVE,2013,1,1,4,0.0
...,...,...,...,...,...,...,...
23201635,54,SEAFOOD,2017,12,31,2,0.0
23201636,54,SEAFOOD,2017,12,31,3,0.0
23201637,54,SEAFOOD,2017,12,31,4,0.0
23201638,54,SEAFOOD,2017,12,31,5,0.0


In [13]:
family_per_store_sales_per_day = pd.get_dummies(family_per_store_sales_per_day, columns=['family'])
family_per_store_sales_per_day

,store_nbr,year,month,day,day_of_week,sales,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,1,2013,1,1,0,0.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2013,1,1,1,0.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,2013,1,1,2,0.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,2013,1,1,3,0.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,2013,1,1,4,0.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23201635,54,2017,12,31,2,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
23201636,54,2017,12,31,3,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
23201637,54,2017,12,31,4,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
23201638,54,2017,12,31,5,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
train, test = train_test_split(family_per_store_sales_per_day, test_size=0.2, random_state=42, shuffle=False)

model = LinearRegression()
features = ['store_nbr', 'year', 'month', 'day', 'day_of_week'] + [col for col in family_per_store_sales_per_day.columns if col.startswith('family_')] 
model.fit(train[features], train['sales'])
predictions = model.predict(test[features])
mse = mean_squared_error(test['sales'], predictions)
print(f'MSE: {mse}')

MSE: 395847.6823950213


# O que deve ter no trabalho?

Faça a previsão de vendas por loja e por produto. Comece mais básico:

1. (FEITO!) Primeiro preveja a venda total de uma Loja ou de um produto em uma loja; 

2. Acrescente a informação de feriados. Use a venda total para confirmar os feriados (locais e nacionais e transferidos)

3. Acrescente a informação do Barril de petróleo

4. Acrescente o pagamento dos funcionários públicos (de 15 em 15 dias)

5. Ao final, faça a previsão para todas as lojas e produtos

6. Submeta e veja o seu resultado.

# 5. Acrescentando a informação de feriados

In [15]:
stores_data

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [16]:
stores_data['city'].value_counts().sort_index()

city
Ambato            2
Babahoyo          1
Cayambe           1
Cuenca            3
Daule             1
El Carmen         1
Esmeraldas        1
Guaranda          1
Guayaquil         8
Ibarra            1
Latacunga         2
Libertad          1
Loja              1
Machala           2
Manta             2
Playas            1
Puyo              1
Quevedo           1
Quito            18
Riobamba          1
Salinas           1
Santo Domingo     3
Name: count, dtype: int64

In [17]:
stores_data['state'].value_counts().sort_index()

state
Azuay                              3
Bolivar                            1
Chimborazo                         1
Cotopaxi                           2
El Oro                             2
Esmeraldas                         1
Guayas                            11
Imbabura                           1
Loja                               1
Los Rios                           2
Manabi                             3
Pastaza                            1
Pichincha                         19
Santa Elena                        1
Santo Domingo de los Tsachilas     3
Tungurahua                         2
Name: count, dtype: int64

In [18]:
holidays_data

,date,type,locale,locale_name,transferred
0,2012-03-02,Holiday,Local,Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,False
...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,False
346,2017-12-23,Additional,National,Ecuador,False
347,2017-12-24,Additional,National,Ecuador,False
348,2017-12-25,Holiday,National,Ecuador,False


In [19]:
holidays_data['date'] = pd.to_datetime(holidays_data['date'])

# Converter a data em componentes numéricos
holidays_data['year'] = holidays_data['date'].dt.year
holidays_data['month'] = holidays_data['date'].dt.month
holidays_data['day'] = holidays_data['date'].dt.day
holidays_data['day_of_week'] = holidays_data['date'].dt.day_of_week

In [20]:
holidays_data = holidays_data.drop(['date'], axis=1)
holidays_data

,type,locale,locale_name,transferred,year,month,day,day_of_week
0,Holiday,Local,Manta,False,2012,3,2,4
1,Holiday,Regional,Cotopaxi,False,2012,4,1,6
2,Holiday,Local,Cuenca,False,2012,4,12,3
3,Holiday,Local,Libertad,False,2012,4,14,5
4,Holiday,Local,Riobamba,False,2012,4,21,5
...,...,...,...,...,...,...,...,...
345,Additional,National,Ecuador,False,2017,12,22,4
346,Additional,National,Ecuador,False,2017,12,23,5
347,Additional,National,Ecuador,False,2017,12,24,6
348,Holiday,National,Ecuador,False,2017,12,25,0


In [21]:
holidays_data['locale_name'].value_counts()

locale_name
Ecuador                           174
Quito                              13
Riobamba                           12
Guaranda                           12
Latacunga                          12
Ambato                             12
Guayaquil                          11
Cuenca                              7
Ibarra                              7
Salinas                             6
Loja                                6
Santa Elena                         6
Santo Domingo de los Tsachilas      6
Quevedo                             6
Manta                               6
Esmeraldas                          6
Cotopaxi                            6
El Carmen                           6
Santo Domingo                       6
Machala                             6
Imbabura                            6
Puyo                                6
Libertad                            6
Cayambe                             6
Name: count, dtype: int64

In [22]:
holidays_data.groupby(['locale', 'locale_name']).count()

type  transferred  year  month  day  \
locale   locale_name                                                           
Local    Ambato                            12           12    12     12   12   
         Cayambe                            6            6     6      6    6   
         Cuenca                             7            7     7      7    7   
         El Carmen                          6            6     6      6    6   
         Esmeraldas                         6            6     6      6    6   
         Guaranda                          12           12    12     12   12   
         Guayaquil                         11           11    11     11   11   
         Ibarra                             7            7     7      7    7   
         Latacunga                         12           12    12     12   12   
         Libertad                           6            6     6      6    6   
         Loja                               6            6     6      6    6   
         Machala                            6            6     6      6    6   
         Manta                              6            6     6      6    6   
         Puyo                               6            6     6      6    6   
         Quevedo                            6            6     6      6    6   
         Quito                             13           13    13     13   13   
         Riobamba                          12           12    12     12   12   
         Salinas                            6            6     6      6    6   
         Santo Domingo                      6            6     6      6    6   
National Ecuador                          174          174   174    174  174   
Regional Cotopaxi                           6            6     6      6    6   
         Imbabura                           6            6     6      6    6   
         Santa Elena                        6            6     6      6    6   
         Santo Domingo de los Tsachilas     6            6     6      6    6   

                                         day_of_week  
locale   locale_name                                  
Local    Ambato                                   12  
         Cayambe                                   6  
         Cuenca                                    7  
         El Carmen                                 6  
         Esmeraldas                                6  
         Guaranda                                 12  
         Guayaquil                                11  
         Ibarra                                    7  
         Latacunga                                12  
         Libertad                                  6  
         Loja                                      6  
         Machala                                   6  
         Manta                                     6  
         Puyo                                      6  
         Quevedo                                   6  
         Quito                                    13  
         Riobamba                                 12  
         Salinas                                   6  
         Santo Domingo                             6  
National Ecuador                                 174  
Regional Cotopaxi                                  6  
         Imbabura                                  6  
         Santa Elena                               6  
         Santo Domingo de los Tsachilas            6

In [23]:
holidays_data

,type,locale,locale_name,transferred,year,month,day,day_of_week
0,Holiday,Local,Manta,False,2012,3,2,4
1,Holiday,Regional,Cotopaxi,False,2012,4,1,6
2,Holiday,Local,Cuenca,False,2012,4,12,3
3,Holiday,Local,Libertad,False,2012,4,14,5
4,Holiday,Local,Riobamba,False,2012,4,21,5
...,...,...,...,...,...,...,...,...
345,Additional,National,Ecuador,False,2017,12,22,4
346,Additional,National,Ecuador,False,2017,12,23,5
347,Additional,National,Ecuador,False,2017,12,24,6
348,Holiday,National,Ecuador,False,2017,12,25,0


In [24]:
national_holidays = holidays_data[holidays_data['locale'] == 'National']
national_holidays = national_holidays[['type', 'transferred', 'year', 'month', 'day', 'day_of_week']].drop_duplicates()
national_holidays['is_holiday'] = 1
stores_national_holidays = stores_data[['store_nbr']].merge(national_holidays[['transferred', 'year', 'month', 'day', 'day_of_week', 'is_holiday']], how='cross')
stores_national_holidays

,store_nbr,transferred,year,month,day,day_of_week,is_holiday
0,1,False,2012,8,10,4,1
1,1,True,2012,10,9,1,1
2,1,False,2012,10,12,4,1
3,1,False,2012,11,2,4,1
4,1,False,2012,11,3,5,1
...,...,...,...,...,...,...,...
9337,54,False,2017,12,22,4,1
9338,54,False,2017,12,23,5,1
9339,54,False,2017,12,24,6,1
9340,54,False,2017,12,25,0,1


In [25]:
# Feriados regionais
regional_holidays = holidays_data[holidays_data['locale'] == 'Regional']
regional_holidays['is_holiday'] = 1

# Juntar feriados regionais com lojas do estado correspondente
stores_regional_holidays = pd.merge(stores_data, regional_holidays, how='left', left_on='state', right_on='locale_name')
stores_regional_holidays = stores_regional_holidays.dropna(subset=['is_holiday'])
stores_regional_holidays = stores_regional_holidays[['store_nbr', 'transferred', 'year', 'month', 'day', 'day_of_week', 'is_holiday']]
stores_regional_holidays[['year', 'month', 'day', 'day_of_week', 'is_holiday']] = stores_regional_holidays[['year', 'month', 'day', 'day_of_week', 'is_holiday']].astype(int)
stores_regional_holidays

C:\Users\wpett\AppData\Local\Temp\ipykernel_26648\3178421999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regional_holidays['is_holiday'] = 1


,store_nbr,transferred,year,month,day,day_of_week,is_holiday
4,5,False,2012,11,6,1,1
5,5,False,2013,11,6,2,1
6,5,False,2014,11,6,3,1
7,5,False,2015,11,6,4,1
8,5,False,2016,11,6,6,1
9,5,False,2017,11,6,0,1
16,12,False,2012,4,1,6,1
17,12,False,2013,4,1,0,1
18,12,False,2014,4,1,1,1
19,12,False,2015,4,1,2,1


In [26]:
# Feriados regionais
local_holidays = holidays_data[holidays_data['locale'] == 'Local']
local_holidays['is_holiday'] = 1

# Juntar feriados regionais com lojas do estado correspondente
stores_local_holidays = pd.merge(stores_data, local_holidays, how='left', left_on='city', right_on='locale_name')
stores_local_holidays = stores_local_holidays.dropna(subset=['is_holiday'])
stores_local_holidays = stores_local_holidays[['store_nbr', 'transferred', 'year', 'month', 'day', 'day_of_week', 'is_holiday']]
stores_local_holidays[['year', 'month', 'day', 'day_of_week', 'is_holiday']] = stores_local_holidays[['year', 'month', 'day', 'day_of_week', 'is_holiday']].astype(int)
stores_local_holidays

C:\Users\wpett\AppData\Local\Temp\ipykernel_26648\2462241644.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_holidays['is_holiday'] = 1


,store_nbr,transferred,year,month,day,day_of_week,is_holiday
0,1,False,2012,12,5,2,1
1,1,False,2012,12,6,3,1
2,1,False,2013,12,5,3,1
3,1,False,2013,12,6,4,1
4,1,False,2014,12,5,4,1
...,...,...,...,...,...,...,...
510,54,False,2013,7,3,2,1
511,54,False,2014,7,3,3,1
512,54,False,2015,7,3,4,1
513,54,False,2016,7,3,6,1


In [27]:
# Combinar todos os feriados
all_stores_holidays = pd.concat([stores_national_holidays, stores_regional_holidays, stores_local_holidays], ignore_index=True)
all_stores_holidays = all_stores_holidays.drop_duplicates(subset=['store_nbr', 'year', 'month', 'day', 'day_of_week'])
all_stores_holidays

,store_nbr,transferred,year,month,day,day_of_week,is_holiday
0,1,False,2012,8,10,4,1
1,1,True,2012,10,9,1,1
2,1,False,2012,10,12,4,1
3,1,False,2012,11,2,4,1
4,1,False,2012,11,3,5,1
...,...,...,...,...,...,...,...
9891,54,False,2013,7,3,2,1
9892,54,False,2014,7,3,3,1
9893,54,False,2015,7,3,4,1
9894,54,False,2016,7,3,6,1


In [28]:
train_data

,store_nbr,family,sales,onpromotion,year,month,day,day_of_week
0,1,AUTOMOTIVE,0.000000,0,2013,1,1,1
1,1,BABY CARE,0.000000,0,2013,1,1,1
2,1,BEAUTY,0.000000,0,2013,1,1,1
3,1,BEVERAGES,0.000000,0,2013,1,1,1
4,1,BOOKS,0.000000,0,2013,1,1,1
...,...,...,...,...,...,...,...,...
3000883,9,POULTRY,438.132996,0,2017,8,15,1
3000884,9,PREPARED FOODS,154.552994,1,2017,8,15,1
3000885,9,PRODUCE,2419.729004,148,2017,8,15,1
3000886,9,SCHOOL AND OFFICE SUPPLIES,121.000000,8,2017,8,15,1


In [29]:
# Merge com informações de feriados
train_data_2 = pd.merge(train_data, all_stores_holidays[['store_nbr', 'year', 'month', 'day', 'day_of_week', 'is_holiday']], on=['store_nbr', 'year', 'month', 'day', 'day_of_week'], how='left')
train_data_2['is_holiday'].fillna(0, inplace=True)

# Verificar os dados de treinamento
train_data_2

,store_nbr,family,sales,onpromotion,year,month,day,day_of_week,is_holiday
0,1,AUTOMOTIVE,0.000000,0,2013,1,1,1,1.0
1,1,BABY CARE,0.000000,0,2013,1,1,1,1.0
2,1,BEAUTY,0.000000,0,2013,1,1,1,1.0
3,1,BEVERAGES,0.000000,0,2013,1,1,1,1.0
4,1,BOOKS,0.000000,0,2013,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...
3000883,9,POULTRY,438.132996,0,2017,8,15,1,0.0
3000884,9,PREPARED FOODS,154.552994,1,2017,8,15,1,0.0
3000885,9,PRODUCE,2419.729004,148,2017,8,15,1,0.0
3000886,9,SCHOOL AND OFFICE SUPPLIES,121.000000,8,2017,8,15,1,0.0


In [30]:
store_sales_per_date_2 = train_data_2.groupby(['store_nbr', 'year', 'month', 'day', 'day_of_week', 'is_holiday'])['sales'].sum().reset_index()
store_sales_per_date_2

,store_nbr,year,month,day,day_of_week,is_holiday,sales
0,1,2013,1,1,1,1.0,0.000000
1,1,2013,1,2,2,0.0,7417.147949
2,1,2013,1,3,3,0.0,5873.244141
3,1,2013,1,4,4,0.0,5919.878906
4,1,2013,1,5,5,1.0,6318.785156
...,...,...,...,...,...,...,...
90931,54,2017,8,11,4,1.0,8513.833984
90932,54,2017,8,12,5,0.0,9139.677734
90933,54,2017,8,13,6,0.0,14246.828125
90934,54,2017,8,14,0,0.0,11882.994141


In [31]:
train, test = train_test_split(store_sales_per_date_2, test_size=0.2, random_state=42, shuffle=False)

model = LinearRegression()
features = ['store_nbr', 'year', 'month', 'day', 'day_of_week', 'is_holiday']
model.fit(train[features], train['sales'])
predictions = model.predict(test[features])
mse = mean_squared_error(test['sales'], predictions)
print(f'MSE: {mse}')

MSE: 375630407.7247053


In [32]:
family_per_store_sales_per_day_2 = train_data_2.groupby(['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week', 'is_holiday'])['sales'].sum().reset_index()
family_per_store_sales_per_day_2

C:\Users\wpett\AppData\Local\Temp\ipykernel_26648\1125209606.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  family_per_store_sales_per_day_2 = train_data_2.groupby(['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week', 'is_holiday'])['sales'].sum().reset_index()


,store_nbr,family,year,month,day,day_of_week,is_holiday,sales
0,1,AUTOMOTIVE,2013,1,1,0,0.0,0.0
1,1,AUTOMOTIVE,2013,1,1,0,1.0,0.0
2,1,AUTOMOTIVE,2013,1,1,1,0.0,0.0
3,1,AUTOMOTIVE,2013,1,1,1,1.0,0.0
4,1,AUTOMOTIVE,2013,1,1,2,0.0,0.0
...,...,...,...,...,...,...,...,...
46403275,54,SEAFOOD,2017,12,31,4,1.0,0.0
46403276,54,SEAFOOD,2017,12,31,5,0.0,0.0
46403277,54,SEAFOOD,2017,12,31,5,1.0,0.0
46403278,54,SEAFOOD,2017,12,31,6,0.0,0.0


In [33]:
family_per_store_sales_per_day_2 = pd.get_dummies(family_per_store_sales_per_day_2, columns=['family'])
family_per_store_sales_per_day_2

,store_nbr,year,month,day,day_of_week,is_holiday,sales,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,1,2013,1,1,0,0.0,0.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2013,1,1,0,1.0,0.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,2013,1,1,1,0.0,0.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,2013,1,1,1,1.0,0.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,2013,1,1,2,0.0,0.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46403275,54,2017,12,31,4,1.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
46403276,54,2017,12,31,5,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
46403277,54,2017,12,31,5,1.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
46403278,54,2017,12,31,6,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [34]:
# Dividir os dados em conjuntos de treinamento e teste
train, test = train_test_split(family_per_store_sales_per_day_2, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo incluindo os novos componentes da data
model = LinearRegression()
features = ['store_nbr', 'year', 'month', 'day', 'day_of_week', 'is_holiday'] + [col for col in train_data_2.columns if col.startswith('family_')]
model.fit(train[features], train['sales'])
predictions = model.predict(test[features])
mse = mean_squared_error(test['sales'], predictions)
print(f'MSE: {mse}')

MSE: 208301.46069790205
